In [1]:
from statsmodels.datasets import longley
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
def myPCA(dataset, explainability):
  meanOf = np.mean(dataset, axis=0)
  meanCentered = dataset - meanOf
  # calculate covariance matrix of mean centered matrix: (BT.B)/(n-1)
  covariance = (meanCentered.T @ meanCentered)/(len(meanCentered)-1)
  # eigendecomposition of covariance matrix
  eigvalues, eigvectors = np.linalg.eig(covariance)
  # transform data
  transformed = meanCentered @ eigvectors

  # sorting eigenvalues/eigenvectors descending order (lambda1 ... lambdaN)
  idx_desc = np.argsort(eigvalues)[::-1]
  eigvalues_desc = eigvalues[idx_desc]
  eigvectors_desc = eigvectors[:,idx_desc]

  # calculating trace of the lambda matrix
  total_variance = np.sum(eigvalues)
  var_exp = [(i/total_variance) for i in sorted(eigvalues, reverse=True)]
  # cumulative(incremental sum of variance)
  cum_sum_exp = np.cumsum(var_exp)

  # stop when cumulativeSum >= retentionVariance
  for importance in range(len(eigvalues)):
    if cum_sum_exp[importance] >= explainability:
      break

  # important pcs
  preservedPCA = meanCentered @ eigvectors_desc[:, 0:importance+1]

  # full transformed data
  fullProjection = meanCentered @ eigvectors_desc

  result = {"number": importance+1,
            "PCA": preservedPCA,
            "transformed": fullProjection}
  return result

In [3]:
data = longley.load_pandas().data # load as pandas dataframe
data.head(3) # lookin at our data

,TOTEMP,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,60323.0,83.0,234289.0,2356.0,1590.0,107608.0,1947.0
1,61122.0,88.5,259426.0,2325.0,1456.0,108632.0,1948.0
2,60171.0,88.2,258054.0,3682.0,1616.0,109773.0,1949.0


In [4]:
data = data.drop(columns=["YEAR", "TOTEMP"]) # removing YEAR and TotalEmployment columns
data.head(3)

,GNPDEFL,GNP,UNEMP,ARMED,POP
0,83.0,234289.0,2356.0,1590.0,107608.0
1,88.5,259426.0,2325.0,1456.0,108632.0
2,88.2,258054.0,3682.0,1616.0,109773.0


In [5]:
for k, v in myPCA(data, 0.98).items():
  print(f'{k}:\n {v}')
  print('-'*30)

number:
 1
------------------------------
PCA:
                 0
0   153725.651033
1   128579.168413
2   129860.699646
3   103301.665927
4    58956.256448
5    40892.552531
6    22425.290793
7    24605.871896
8    -9744.178762
9   -31494.761083
10  -55145.582797
11  -57031.826134
12  -95190.353553
13 -115178.327941
14 -130889.707248
15 -167672.419169
------------------------------
transformed:
                 0            1            2           3         4
0   153725.651033   831.133744  -225.988314 -476.261262  0.566255
1   128579.168413   294.987174  -688.833042 -177.897530 -1.615593
2   129860.699646  1892.822103   174.525277  154.556109 -0.093472
3   103301.665927  1144.347197  -200.235534  188.394784  1.389276
4    58956.256448 -1619.149486    39.330844  174.550842 -0.159970
5    40892.552531 -1983.519868   361.135629   -3.761486 -0.392042
6    22425.290793 -1637.341370   398.270516 -469.648882 -0.666309
7    24605.871896   333.711983  1035.270169  115.563662  0.007262
8    -9

In [6]:
# Using R scaling:
dataset = longley.load_pandas().data
dataset.iloc[:, 2] = dataset.iloc[:, 2]/1000
dataset.iloc[:, 3] = dataset.iloc[:, 3]/10
dataset.iloc[:, 4] = dataset.iloc[:, 4]/10
dataset.iloc[:, 5] = dataset.iloc[:, 5]/1000
dataset = dataset.drop(['TOTEMP','YEAR'], axis = 1)
for k, v in myPCA(dataset, 0.98).items():
  print(f'{k}:\n {v}')
  print('-'*30)

number:
 3
------------------------------
PCA:
              0           1          2
0  -186.637681   77.409745 -22.328437
1  -171.161527   77.394341 -46.866466
2   -84.573098  144.528213  34.780831
3   -84.534362  114.238901   4.842686
4  -106.029319  -81.237768   7.962879
5   -94.849817 -133.814948  20.982680
6   -85.391818 -139.830937   4.368918
7    17.007506  -25.385423  81.729325
8    -3.841327  -52.910817   7.945521
9     4.792683  -50.452892 -20.928929
10   29.265157  -47.130117 -32.180049
11  137.751035   65.416981  47.092654
12  111.094025    9.163444 -24.960414
13  133.188654   12.399691 -32.508112
14  200.996533   53.720504   7.914330
15  182.923355  -23.508919 -37.847417
------------------------------
transformed:
              0           1          2         3         4
0  -186.637681   77.409745 -22.328437 -1.666480  0.163309
1  -171.161527   77.394341 -46.866466  1.595662  0.426566
2   -84.573098  144.528213  34.780831 -0.092104  0.077903
3   -84.534362  114.238901   

In [7]:
# Using built-in function in python
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(dataset)
X_new = pca.transform(dataset)
X_new

array([[-1.86637681e+02,  7.74097447e+01, -2.23284369e+01,
         1.66647980e+00, -1.63309053e-01],
       [-1.71161527e+02,  7.73943407e+01, -4.68664665e+01,
        -1.59566185e+00, -4.26566116e-01],
       [-8.45730975e+01,  1.44528213e+02,  3.47808310e+01,
         9.21040574e-02, -7.79029312e-02],
       [-8.45343621e+01,  1.14238901e+02,  4.84268648e+00,
         9.80367072e-01,  4.76986919e-01],
       [-1.06029319e+02, -8.12377681e+01,  7.96287854e+00,
        -8.87103754e-01,  1.63369002e-01],
       [-9.48498168e+01, -1.33814948e+02,  2.09826803e+01,
        -5.21482726e-01, -6.59274306e-02],
       [-8.53918178e+01, -1.39830937e+02,  4.36891845e+00,
         4.15696974e-01, -4.79109296e-01],
       [ 1.70075059e+01, -2.53854226e+01,  8.17293255e+01,
         6.04319996e-01, -1.67944625e-01],
       [-3.84132689e+00, -5.29108168e+01,  7.94552109e+00,
         1.52061136e+00,  7.26193103e-01],
       [ 4.79268313e+00, -5.04528920e+01, -2.09289292e+01,
        -8.13706699e-03